### 輸入英文遊戲名稱後即可得到其在 API 中該評論作者的總留言數
最終輸出會是字典 review_id_2_num_reviews，key 是評論 id，value 是 該評論作者的總留言數

get steam_reviews.csv: https://www.kaggle.com/najzeko/steam-reviews-2021

In [14]:
import numpy as np
import pandas as pd

col_list = ["app_id", "app_name"]
# the file downloaded on kaggle
steam_data = pd.read_csv('steam_reviews.csv', usecols=col_list)

In [15]:
# get the game id and name pair
# count the # of reviews of each game
review_by_games = steam_data.groupby(["app_id", "app_name"]).size().sort_values(ascending=False)

# top 20 game with most reviews
review_by_games[:20]

app_id  app_name                      
578080  PLAYERUNKNOWN'S BATTLEGROUNDS     1644255
271590  Grand Theft Auto V                1019116
359550  Tom Clancy's Rainbow Six Siege     841918
105600  Terraria                           672815
4000    Garry's Mod                        655524
252490  Rust                               549074
252950  Rocket League                      498565
218620  PAYDAY 2                           487747
945360  Among Us                           485293
292030  The Witcher 3: Wild Hunt           469395
381210  Dead by Daylight                   418897
346110  ARK: Survival Evolved              400009
227300  Euro Truck Simulator 2             387553
413150  Stardew Valley                     315717
72850   The Elder Scrolls V: Skyrim        294966
431960  Wallpaper Engine                   292790
582010  Monster Hunter: World              290946
367520  Hollow Knight                      269854
242760  The Forest                         239734
322330  Don

### Enter the games you want to get number of reviews done by a review's author

In [16]:
steam_app_name_id = steam_data.drop_duplicates()

search_app_names = input("for example: The Legend of Heroes: Trails of Cold Steel II, Rapture Rejects, BERSERK and the Band of the Hawk").split(',')
search_app_names = [i.strip() for i in search_app_names]

for example: The Legend of Heroes: Trails of Cold Steel II, Rapture Rejects, BERSERK and the Band of the Hawk PLAYERUNKNOWN'S BATTLEGROUNDS, Grand Theft Auto V, Tom Clancy's Rainbow Six Siege 


In [17]:
name_2_id = {}
for ID, name in zip(steam_app_name_id.app_id, steam_app_name_id.app_name): 
    name_2_id[name] = ID

### Call Steam API

In [42]:
import requests
import json
# provide a valid, URL encoded cursor
import urllib.parse

# how many reviews a review's author made
review_id_2_num_reviews = {}
# give up at the number if the new reviews don't show in the next n page
cursor_tolerate = 5


for name in search_app_names:
    try:
        print('Name:', name, ', id:', name_2_id[name])
    except:
        print('')
        break
    
    # API parameter setting
    init_url = "https://store.steampowered.com/appreviews/" \
    +str(name_2_id[name])\
    + "?json=1&num_per_page=100&purchase_type=all&language=all"
    
    counter = 1
    # lead to next page
    cursor = ''
    review_cnt = 0
    

    cursor_stuck_cnt = 0
    while True:
        if counter == 1:
            response = requests.get(init_url)
        else:
            response = requests.get(init_url + '&cursor=' + cursor)
        # print(response.status_code)

        try:
            cursor = urllib.parse.quote(response.json()['cursor'], safe="")
            for i in response.json()['reviews']:
                review_id_2_num_reviews[int(i['recommendationid'])] = int(i['author']['num_reviews'])

            
            # if there are new reviews from the this page, then go to next page
            if len(review_id_2_num_reviews) > review_cnt:
                review_cnt = len(review_id_2_num_reviews)
                print('accumulative reviews: ', len(review_id_2_num_reviews))
            # if not, simply break and enter the next game
            else:  
                print('Same reviews...')
                cursor_stuck_cnt += 1

                if cursor_stuck_cnt >= cursor_tolerate:
                    print('No new reviews!')
                    break
                
            counter += 1
            # print which page we are on every 10 pages
            if counter % 10 == 0:
                print('has loaded ', counter, 'pages of reviews......')
        except:
            print('Load error!')
            print(cursor)
            print(json.dumps(response.json(), indent=4, sort_keys=True))
            break  
    print()  # between games

Name: PLAYERUNKNOWN'S BATTLEGROUNDS , id: 578080
accumulative reviews:  100
accumulative reviews:  200
accumulative reviews:  300
accumulative reviews:  400
accumulative reviews:  500
accumulative reviews:  600
accumulative reviews:  700
accumulative reviews:  800
accumulative reviews:  900
has loaded  10 pages of reviews......
accumulative reviews:  1000
accumulative reviews:  1100
accumulative reviews:  1200
accumulative reviews:  1300
accumulative reviews:  1400
accumulative reviews:  1500
accumulative reviews:  1600
accumulative reviews:  1700
accumulative reviews:  1800
accumulative reviews:  1900
has loaded  20 pages of reviews......
accumulative reviews:  1994
accumulative reviews:  2069
Same reviews...
Same reviews...
Same reviews...
Same reviews...
Same reviews...
No new reviews!

Name: Grand Theft Auto V , id: 271590
accumulative reviews:  2169
accumulative reviews:  2269
accumulative reviews:  2369
accumulative reviews:  2469
accumulative reviews:  2569
accumulative reviews:

### Final output

In [44]:
len(review_id_2_num_reviews)

5456

In [46]:
# review_id_2_num_reviews